# Preprocessing of data


In [1]:
import pandas as pd
import essay
import numpy as np
import pickle
import re

In [2]:
def mbti_to_big5(mbti):
    # check https://en.wikipedia.org/wiki/Myers%E2%80%93Briggs_Type_Indicator
    # in mbti (myers briggs) ther is invrovert vs. extrovert
    # which corellates with Extroversion in BIG FIVE
    mbti = mbti.lower()
    cEXT, cNEU, cAGR, cCON, cOPN = 0,np.NaN,0,0,0
    if mbti[0] == "i":
        cEXT = 0
    elif mbti[0] == "e":
        cEXT = 1

    # in mbti (myers briggs) ther is I*N*TUITION vs SENSING
    # which corellates with OPENNESS in BIG FIVE
    if mbti[1] == "n":
        cOPN = 1
    elif mbti[1] == "s":
        cOPN = 0   

    # in mbti (myers briggs) ther is THINKER vs FEELER
    # which corellates with AGREEABLENESS in BIG FIVE
    if mbti[2] == "t":
        cAGR = 0
    elif mbti[2] == "f":
        cAGR = 1

    # in mbti (myers briggs) ther is JUDGER vs PERCEIVER
    # which corellates with CONSCIENTIOUSNESS in BIG FIVE (worst corellation)
    # especially bec. orderlyness corellates with conscientiousness
    if mbti[3] == "p":
        cCON = 0
    elif mbti[3] == "j":
        cCON = 1

    return cEXT, cNEU, cAGR, cCON, cOPN

In [3]:
def remove_unemotional_scentences(emotional_words, text_as_one_string):
    reduced_s = ""
    scentences = re.split('(?<=[.!?]) +', text_as_one_string)
    for s in scentences:
        if any(e in s for e in emotional_words):
            reduced_s = reduced_s + s + " "
        else:
            pass
    return reduced_s

In [4]:
# simply put every row of our read dataframe into a list of 
# the object "Essay"
# remove data from list substract
def create_essays(df, subtract=None):
    essays = []
    for index, row in df.iterrows():
        essays.append(essay.Essay(row.TEXT, row.cEXT, row.cNEU, row.cAGR, row.cCON, row.cOPN))  

    # remove scentences which do not contain emotionally charged words 
    # from the emotional lexicon
    if subtract != None:
        for x in essays:
            x.filtered_text = remove_unemotional_scentences(emotional_words, x.clean_text)

    return essays

# Loading the essays from paper
## scientific gold standard
## https://sentic.net/deep-learning-based-personality-detection.pdf
### (scientific gold standard "stream of counsciousness" essays labeled with personality traits of the big five)

In [5]:
# we read in the data from "essays.csv" and 
# "essays.csv" contains all essays with classification 
df_essays = pd.read_csv('data/training/essays.csv', encoding='cp1252', delimiter=',', quotechar='"')

# for every essay, we replace the personalitiy categories 
# of the essay wich are "y" and "n" with "1" and "0" 
for e in df_essays.columns[2:7]:
    df_essays[e] = df_essays[e].replace('n', '0')
    df_essays[e] = df_essays[e].replace('y', '1')
    # not sure if we need this line: furter investigation possible:
    df_essays[e] = pd.to_numeric(df_essays[e])

df_essays = df_essays[["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_essays

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,I can't believe it! It's really happening! M...,1,0,1,1,0
4,"Well, here I go with the good old stream of co...",1,0,1,0,1
...,...,...,...,...,...,...
2462,I'm home. wanted to go to bed but remembe...,0,1,0,1,0
2463,Stream of consiousnesssskdj. How do you s...,1,1,0,0,1
2464,"It is Wednesday, December 8th and a lot has be...",0,0,1,0,0
2465,"Man this week has been hellish. Anyways, now i...",0,1,0,0,1


# Load the MBTI kaggle dataset
## https://www.kaggle.com/datasnaek/mbti-type

In [7]:
df_kaggle = pd.read_csv('data/training/mbti_1.csv',  skiprows=0 )

df_kaggle["cEXT"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[0], 1)
df_kaggle["cNEU"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[1], 1)
df_kaggle["cAGR"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[2], 1)
df_kaggle["cCON"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[3], 1)
df_kaggle["cOPN"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[4], 1)

df_kaggle = df_kaggle[["posts", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_kaggle.columns = ["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

# remove som fancy ||| things
df_kaggle["TEXT"] = df_kaggle.apply(lambda x: x.TEXT.replace("|||", " ")[:], 1)


df_kaggle

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,'http://www.youtube.com/watch?v=qsXHcwe3krw ht...,0,NaN,1,1,1
1,'I'm finding the lack of me in these posts ver...,1,NaN,0,0,1
2,'Good one _____ https://www.youtube.com/wat...,0,NaN,0,0,1
3,"'Dear INTP, I enjoyed our conversation the o...",0,NaN,0,1,1
4,'You're fired. That's another silly misconcept...,1,NaN,0,1,1
...,...,...,...,...,...,...
8670,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,NaN,1,0,0
8671,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


# Load Reddit Dataset
## Matej Gjurković  https://peopleswksh.github.io/pdf/PEOPLES12.pdf
### received on request


In [8]:
# the file is kinda huge. thanks Matej
file = 'data/training/typed_comments.csv'
df_reddit = pd.read_csv(file)

c:\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3050: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
#remove some rows to to keep longer text (the 420 because it makes avg word count compareable to the rest of the data)
df_reddit = df_reddit[df_reddit.word_count > 420]

df_reddit["cEXT"] =   df_reddit.apply(lambda x: mbti_to_big5(x.type)[0], 1)
df_reddit["cNEU"] =   df_reddit.apply(lambda x: mbti_to_big5(x.type)[1], 1)
df_reddit["cAGR"] =   df_reddit.apply(lambda x: mbti_to_big5(x.type)[2], 1)
df_reddit["cCON"] =   df_reddit.apply(lambda x: mbti_to_big5(x.type)[3], 1)
df_reddit["cOPN"] =   df_reddit.apply(lambda x: mbti_to_big5(x.type)[4], 1)
df_reddit = df_reddit[["comment", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_reddit.columns = ["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]
df_reddit.reset_index(drop=True, inplace=True)
df_reddit

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Okay, this has been educational. I was not awa...",1,NaN,0,0,1
1,This depends on *what MBTI means to you*.The *...,1,NaN,0,0,1
2,O witty! Gotta do the TYPE_MENTION PIT maneuve...,1,NaN,0,0,1
3,I'm glad it's interesting! :)So I had to spend...,1,NaN,0,0,1
4,"Yeah, I don't *like* to dismiss a cool new ide...",1,NaN,0,0,1
...,...,...,...,...,...,...
78217,"People post videos, memes, and curse in PPD pr...",1,NaN,1,1,0
78218,"Well first off, I'm not a libertarian in the s...",1,NaN,1,1,0
78219,"Yeah, I am kinda ending Westphalian nation sta...",1,NaN,1,1,0
78220,"Hey No problem, was just a heads up for the f...",1,NaN,1,1,0


# Load Emotional Lexicon to substract from data

In [10]:
# also from "Emotional_Lexicon.csv" we read in the data, which is a list of words and 
# has several categories of emotions. 
# anger - anticipation - disgust - fear - joy - negative - positive 
# - sadness - surprise - trust - Charged
df_lexicon = pd.read_csv('data/training/Emotion_Lexicon.csv', index_col=0)


# some of the words have no emotional category, 
# so let's remove them as they have no use to us.
# can be improved by not even loading them when all columns are 0. maybe later.
df_lexicon = df_lexicon[(df_lexicon.T != 0).any()]
emotional_words = df_lexicon.index.tolist()

# Concatinate the datasets for 3 different data to work with and compare to

## Create Data Base 1 - only esseys.csv - and save as object list

In [11]:
# save preprocessed data by converting into OBJECT essay and save with pickle and removing non emotional scentences
essays = create_essays(df_essays, emotional_words)
pickle.dump(essays, open( "essays/essays2467.p", "wb"))
print("saved entries: ", len(essays))

saved entries:  2467


## Create Data Base 2 - Essay Data and Kaggle data - and save as object list

In [12]:
# concatinate the dataframes:
frames  = [df_essays, df_kaggle]
essays_kaggle = pd.concat(frames, sort=False)
essays_kaggle.reset_index(drop=True)

# preprocess data by converting into OBJECT essay and save with pickle and removing non emotional scentences
essays_kaggle = create_essays(essays_kaggle, emotional_words)
pickle.dump(essays_kaggle, open("essays/essays11142.p", "wb"))
print("saved entries: ", len(essays_kaggle))

saved entries:  11142


## Create Data Base 3 - Essay Data and Kaggle data and Reddit data - and save as object list

In [13]:
# concatinate the dataframes:
frames  = [df_essays, df_kaggle, df_reddit]
essays_kaggle_reddit = pd.concat(frames, sort=False)
essays_kaggle_reddit.reset_index(drop=True)

# preprocess data by converting into OBJECT essay and save with pickle and removing non emotional scentences
essays_kaggle_reddit = create_essays(essays_kaggle_reddit, emotional_words)
pickle.dump(essays_kaggle_reddit, open("essays/essays89364.p", "wb"))
print("saved entries: ", len(essays_kaggle_reddit))

saved entries:  89364
